In [1]:
#pip install mlxtend

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [3]:
def splitdata(indepX,depY):
    x_train,x_test,y_train,y_test=train_test_split(indepX,depY,test_size=0.30,random_state=0)
    from sklearn.preprocessing import StandardScaler
    sc=StandardScaler()
    x_train=sc.fit_transform(x_train)
    x_test=sc.transform(x_test)
    return x_train,x_test,y_train,y_test

In [4]:
def rscorecal(regressor,x_test,y_test):
    from sklearn.metrics import r2_score
    y_pred=regressor.predict(x_test)
    rscore=r2_score(y_test,y_pred)
    return rscore

In [5]:
def linear(x_train,y_train,x_test):
    from sklearn.linear_model import LinearRegression
    lr=LinearRegression()
    sbs = SFS(lr, k_features=3, forward=False, floating=False, scoring='r2')
    sbs=sbs.fit(x_train,y_train)
   # Get the selected features
    selfea_list = sbs.k_feature_idx_
    print("Selected feature indices:", selfea_list)
    lr.fit(x_train[:, selfea_list], y_train)
    # Calculate R-squared score on the test set using selected features (NumPy indexing)
    R_score = rscorecal(lr, x_test[:, selfea_list], y_test)
    coefficients = lr.coef_
    # Display feature importance
    features = [f'Feature {i}' for i in range(x_train[:, selfea_list].shape[1])]
    importance_df = pd.DataFrame({'Feature': features, 'Importance': coefficients})
    importance_df.sort_values(by='Importance', ascending=False, inplace=True)
    print(importance_df)
    return R_score

In [6]:
dataset=pd.read_csv('prep.csv')

In [7]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [8]:
indepX=dataset.drop('classification_yes',axis=1)
depY=dataset['classification_yes']

In [9]:
def selectkregressor(rsclin):
    df=pd.DataFrame(index=['chi_square'],columns=['Linear'])
    for num,idex in enumerate(df.index):
        df['Linear'][idex]=rsclin[num]
        #df['RandomForest'][idex]=rsclin[num]
        return df

In [10]:
x_train,x_test,y_train,y_test=splitdata(indepX,depY)

In [11]:
rsclin=[]
#rsrfr=[]

In [12]:
    rsclin.append(linear(x_train,y_train,x_test))
    #rsrfr.append(Random(x_train,y_train,x_test,indepX))

Selected feature indices: (9, 14, 15)
     Feature  Importance
1  Feature 1    0.156524
2  Feature 2    0.145964
0  Feature 0   -0.275142


In [13]:
result=selectkregressor(rsclin)
result

C:\Users\Admin\AppData\Local\Temp\ipykernel_7944\2726242617.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['Linear'][idex]=rsclin[num]


,Linear
chi_square,0.683544
